In [1]:
from deeppavlov.models.embedders.elmo_embedder import ELMoEmbedder
elmo = ELMoEmbedder("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-wiki_600k_steps.tar.gz")

[nltk_data] Downloading package punkt to /home/alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/alex/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/alex/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
W0409 19:09:15.079932 140353324242688 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14
Using TensorFlow backend.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0409 19:12:48.741482 140353324242688 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [2]:
import pandas as pd
import numpy as np
from progressbar import progressbar as pb

df = pd.read_csv('all_data_fixed3.csv')

In [3]:
print(len(df))
df.head()

5330


,0,1,2,3,4,5
0,Совещательный орган при императоре в начале XX в,Сенат,Государственный совет,Консилия министров,Верховный тайный совет,2
1,Министр внутренних дел с 1904 г либерал автор ...,Лорис Меликов,Святополк Мирский,Милюков,Витте,2
2,Какое из названных событий произошло 27 января...,подрыв флагманского корабля Петропавловск,высадка японских войск на Ляодунском полуострове,героический бой Варяга и Корейца,заключение Портсмутского мирного договора,3
3,Что из названного относится к результатам перв...,появление многопартийности,формирование конституционной монархии,ликвидация сословного строя,введение рабочего контроля за производством,1
4,Для выхода из экономического кризиса вызванног...,ввело 8 часовой рабочий день в промышленности,ликвидировало помещичье землевладение,отменило выкупные платежи,ввело продовольственную разверстку,4


In [4]:
def get_vec(lst):
    res = np.zeros((5,300))
    for i in range(len(lst)-1):
        cur_vec = np.zeros(300)
        cur_tag = lst[i].split(' ')
        for c in cur_tag:
            try:
                cur_vec += fasttext.get_word_vector(c)
            except:
                continue
        res[i,:] = cur_vec
    return res

In [5]:
lst_df = df.values.tolist()
lst_df[0]

['Совещательный орган при императоре в начале XX в',
 'Сенат',
 'Государственный совет',
 'Консилия министров',
 'Верховный тайный совет',
 2]

In [6]:
for i in range(len(lst_df)):
    for j in range(5):
        lst_df[i][j] = lst_df[i][j].split(' ')
    lst_df[i] = lst_df[i][:-1]

In [7]:
lst_df[1]

[['Министр',
  'внутренних',
  'дел',
  'с',
  '1904',
  'г',
  'либерал',
  'автор',
  'проекта',
  'О',
  'мерах',
  'к',
  'усовершенствованию',
  'государственного',
  'порядка'],
 ['Лорис', 'Меликов'],
 ['Святополк', 'Мирский'],
 ['Милюков'],
 ['Витте']]

In [8]:
ELMO_embs = []
from progressbar import progressbar as pb

for i in pb(range(len(lst_df))):
    try:
        ELMO_embs.append(elmo(lst_df[i]).tolist())
    except:
        print(lst_df[i])
        break

100% (5330 of 5330) |####################| Elapsed Time: 0:09:47 Time:  0:09:47


In [9]:
ELMO_embs_np = np.array(ELMO_embs)

In [10]:
ELMO_embs_np.shape

(5330, 5, 1024)

In [12]:
import pickle

with open('elmo_embed3.pickle', 'wb') as f:
    pickle.dump(ELMO_embs, f)